# To Handle New Data Requests Automatically
- beginning of notebook is assumed to be interactive until the requests have been checked
- all progress and exception logging is done only for main loop
- copy and paste the e-mail response and send from gcs.cmip6.ldeo@gmail.com account

- `gsurl` is the GCS url for the dataset
- `ds_dir` is our dataset identifier ( activity_id/institution_id/.../variable_id/grid_label ) NO version included

In [ ]:
import pandas as pd
import gcsfs
import xarray as xr
from datetime import datetime
import itertools
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials

### Local modules

In [ ]:
import myconfig
from mydataset import dir2url_, dir2dict
from mytasks import Check, Download, ReadFiles, SaveAsZarr, Upload, Cleanup
from mysearch import esgf_search
#from myidentify import gsurl2search, gsurl2dsdir
from myutilities import search_df, remove_from_GC_bucket
from myrequest import requests, set_request_id
from myresponse import response, get_details, dict_to_dfcat

In [ ]:
# Directly connect to Search Status Page for updating
# Anyone can view: https://docs.google.com/spreadsheets/d/1yAt7604tVt7OXXZUyL2uALtGP2WVa-Pb5NMuTluFsAc/edit?usp=sharing

json_keyfile = '/home/naomi/json/CMIP6-d0cb1df722d1.json'
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
gc = gspread.authorize(credentials)
sheet_name = "CMIP6_UpdateSchedule"
sh = gc.open(sheet_name)
wks = sh.worksheet("Searches")

# read the first row to get the column labels
columns = wks.row_values(3)
col_status = columns.index('status')
col_run = columns.index('last run')
col_drive = columns.index('HD')
col_dataset = columns.index('current dataset')
col_node = columns.index('search node')

In [ ]:
# CONFIGURE ESGF Search here
node_pref = myconfig.node_pref
dtype = myconfig.dtype
hd = '/h123'
myconfig.local_target_prefix = hd + '/naomi/zarr-minimal/'
dir2local = dir2url_(myconfig.local_target_prefix)

update_ESGF = True
update_Needed = True

search_node = 'llnl'
#search_node = 'dkrz'
ESGF_site = dtype[search_node]

print('zarrs will be written to: ',myconfig.local_target_prefix)

In [ ]:
# CONFIGURE GCS
fs     = gcsfs.GCSFileSystem(token='anon', access='read_only',cache_timeout=-1)
df_GCS = pd.read_csv('https://cmip6.storage.googleapis.com/pangeo-cmip6-noQC.csv', dtype='unicode')
df_GCS['ds_dir'] = df_GCS.apply(lambda row: myconfig.target_format % row,axis=1)

# make available to all modules (except those)
myconfig.fs = fs
myconfig.df_GCS = df_GCS

### Get new Google Sheet requests
- by default, only the new rows from the sheet are considered
- specifying a list of rows or emails will add older entries 

In [ ]:
df_prior = pd.read_csv('csv/requests.csv', dtype='unicode', encoding='latin1')

rows = []   
emails = []
#rows = [4] #range(0,5) #[0,5,9]   # line number - 2
#emails = ['c.wang@princeton.edu']

df_request_new, dtrouble = requests(df_prior,rows=rows,emails=emails)
request_id = set_request_id()

# Check for mal-formed requests (non-existent variables, etc)
if len(dtrouble)>=1:
    print(dtrouble)

# print all active requests:
display(df_request_new)

In [ ]:
# by index
#type = 'input'
type = 'last'

if type == 'input':
    resp = input('index?  (after entering number, click on next cell to advance)')
    df_request_new = df_request_new.loc[[int(resp)]]
else:
    timestamps = df_request_new.Timestamp.unique()
    #df_request_new = df_request_new[df_request_new.Timestamp == timestamps[-1]]
    df_request_new = df_request_new.loc[[3]]

df_request_new

In [ ]:
asearch = {}
search_keys = {'table':'table_id','experiments':'experiment_id','variables':'variable_id','models':'source_id','members':'member_id'}

for item, row in df_request_new.iterrows():
    asearch['table_id'] = [row.table]
    request = item
    for key in search_keys.keys():
        if key == 'table':
            continue
        klist = row[key]
        if not ('All' in klist)|('One' in klist):  # Note, we no longer get just one member_id without specifying which
            asearch[search_keys[key]] = klist

label = f'request-{request}'

try:
    search_row = wks.find(label).row
    print(search_row)
except:
    print('need another row in Google Form')
    wks.append_row([label])
    search_row = wks.find(label).row
    
date = datetime.now().strftime('%Y-%m-%d')
wks.update_cell(search_row, col_run + 1, date)
wks.update_cell(search_row, col_drive + 1, hd)
wks.update_cell(search_row, col_node + 1, search_node)

col_activity   = columns.index('activity_id')
col_table      = columns.index('table_id')
col_experiment = columns.index('experiment_id')
col_variable   = columns.index('variable_id')

for key in asearch.keys():
    vals = asearch[key]
    if key in columns:
        col = columns.index(key)
        if len(vals) == 1:
            wks.update_cell(search_row, col + 1, f'{vals[0]}')
        else:
            wks.update_cell(search_row, col + 1, f'{vals}')            
        
label, asearch

In [ ]:
# make dataframe of ESGF search results

if update_ESGF:
    x = [value for key,value in asearch.items()]
    searches = [p for p in itertools.product(*x)]

    dESGF = []
    for s in searches:
        dsearch = dict(zip(asearch.keys(),s))
        print(dsearch)
        df = esgf_search(dsearch, server=ESGF_site)
        if len(df)>0:
            dESGF += [df]

    df_ESGF = pd.concat(dESGF)
    df_ESGF.to_csv(f'csv/ESGF_{label}.csv',index=False)
else:
    df_ESGF = pd.read_csv(f'csv/ESGF_{label}.csv', dtype='unicode')

len(df_ESGF), len(df_ESGF.ds_dir.unique())
#df_ESGF.ds_dir.unique()

In [ ]:
# make dataframe of all needed

if update_Needed:
    df_ESGF['cversion'] = [int(s[1:]) for s in df_ESGF.version_id]
    df_ESGF = df_ESGF.sort_values(by=['cversion'])
    df_ESGF = df_ESGF.drop_duplicates(subset =["ds_dir","ncfile"],keep='last')
 
    df_GCS['ds_dir'] = df_GCS.apply(lambda row: myconfig.target_format % row,axis=1)
    df_GCS = df_GCS[df_GCS.version != 'unknown']
    df_GCS['cversion'] = [int(s) for s in df_GCS.version]
    df_GCS = df_GCS.sort_values(by=['version'])
    df_GCS = df_GCS.drop_duplicates(subset =["ds_dir"],keep='last')

    df = pd.merge(df_ESGF, df_GCS, how='outer', on=['ds_dir'], indicator=True, suffixes=('', '_y'),)
    df_new_dataset = df[df._merge == 'left_only'] 
    
    df_check = df[df._merge == 'both'] # we might want to add these if the ESGF version is newer than the GCS version
    # New versions (at least 2 days newer) which exist at ESGF
    if len(df_check) > 0:
        df_check['dversion'] = df_check.apply(lambda row: row.cversion - row.cversion_y,axis=1)
        df_check = df_check.sort_values(by=['dversion'])
        df_new_version = df_check[df_check.dversion > 2] # at least 2 days newer

        # Want to append together df_new_dataset and df_new_version
        keys = ['activity_id', 'institution_id', 'source_id', 'experiment_id', 'member_id', 'table_id', 'variable_id', 'grid_label', 'version_id', 'ncfile', 'file_size', 'url', 'data_node', 'ds_dir', 'node_order', 'start', 'stop']
        df_new_version = df_new_version[keys]
        df_new_dataset = df_new_dataset[keys]

        df_needed = df_new_dataset.append(df_new_version)
    else:
        df_needed = df_new_dataset
    df_needed['version'] = [s[1:] for s in df_needed.version_id]

    num_stores = 0
    if len(df_needed) > 0:
        num_stores = df_needed.ds_dir.nunique() 
        print(f'needed: nfiles={len(df_needed)}, nstores={num_stores}')
    
        df_needed['member'] = [int(s.split('r')[-1].split('i')[0]) for s in df_needed['member_id']]
        df_needed = df_needed.sort_values(by=['member'])
        #df_needed['zsize'] = [df_needed[df_needed.ds_dir==zs]['file_size'].sum() for zs in df_needed['ds_dir']]
        #df_needed = df_needed.sort_values(by=['zsize'])
        df_needed.to_csv(f'csv/needed_{label}.csv',index=False)
else:
    df_needed = pd.read_csv(f'csv/needed_{label}.csv', dtype='unicode')

print('Variables')
try:
    for var in df_needed.variable_id.unique():
        print(var,df_needed[df_needed.variable_id==var].ds_dir.nunique())

    print('\nExperiments')
    for exp in df_needed.experiment_id.unique():
        print(exp,df_needed[df_needed.experiment_id==exp].ds_dir.nunique())
except:
    print('no new data available')

In [ ]:
# make available to all modules
myconfig.df_needed = df_needed
ds_dirs = df_needed.ds_dir.unique()
numdsets = len(ds_dirs)

In [ ]:
progress_log  = f'logs/progress_{label}.log'
failure_log   = f'logs/failure_{label}.log'
success_log   = f'logs/success_{label}.log'
logs = {1:progress_log, 2:failure_log, 3:success_log}

### ierr:
- 0 : proceed with next task
- 1 : write to progress_log, go to next dataset (finished or try again)
- 2 : write to failure_log,  go to next dataset (mark as un-usable - do not try again until problem is solved) 
- 3 : write to success_log,  go to next dataset (dataset added to cloud)

In [ ]:
def write_log(file,str,verbose=True):
    date = datetime.now().strftime('%Y%m%d-%H%M')
    f = open(file,'a')
    if verbose:
        print(str)
    f.write(f'{date}:{str}\n')
    f.close()
    return

In [ ]:
# refresh the gcsfs
fs.invalidate_cache()
date = str(datetime.now().strftime("%Y%m%d-%H%M"))
for log in [1,2,3]:   
    write_log(logs[log],f'starting at {date}') 
zdict = {}

for item, ds_dir in enumerate(ds_dirs):
    #if item > 0:
    #    continue
    print(f'\n{item}/{numdsets-1}',ds_dir)

    #skip = 'EC-Earth3'
    skip = 'none'
    if skip in ds_dir:
        write_log(progress_log,f'{ds_dir} skipping {skip}') 
        continue

    version = df_needed[df_needed.ds_dir==ds_dir].version.values[0]
        
    (ierr, exc) = Check(ds_dir, version, dir2local)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir} {exc}'); continue
                     
    (gfiles, version, ierr, exc) = Download(ds_dir)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir}, {exc}'); continue
        
    search_row = wks.find(label).row
    wks.update_cell(search_row, col_dataset + 1, ds_dir)
    
    (ds,ierr,exc) = ReadFiles(ds_dir, gfiles, version, dir2dict)
    
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir}, {exc}'); continue

    (version,ierr,exc) = SaveAsZarr(ds_dir, ds, dir2local)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir}, {exc}'); continue
        
    (zbdir, gsurl, ierr,exc) = Upload(ds_dir, version, dir2local)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir}, {exc}'); continue

    (ierr,exc) = Cleanup(ds_dir, version, gfiles, dir2local)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir}, {exc}'); continue

    vlist = get_details(ds_dir, ds, dir2local)
    zdict[item] = vlist

    date = datetime.now().strftime('%H:%M, %b%d')
    status_str = f'{item+1} of {numdsets} at {date}'
    search_row = wks.find(label).row
    wks.update_cell(search_row, col_status + 1, status_str)

    write_log(success_log,f'{zbdir} saved to {gsurl}'); continue

In [ ]:
#gs://cmip6/CMIP/CCCma/CanESM5/piControl/r1i1p1f1/Omon/mlotst/gn/
assert False

### Make a table of acquired data to send in email to requestor

In [ ]:
if len(zdict) == 0 :
    print('nothing else to do')
    exit
else:
    dz = dict_to_dfcat(zdict)

In [ ]:
try:
    df_master_new = pd.concat([df_GCS, dz],sort=True)
except:
    df_master_new = df_GCS

In [ ]:
ldict = []
names = ""
print('Re: CMIP6 GCS Data Request (Responses)')
for row in df_request_new.values:
    rdict = dict(zip(df_request_new.keys(),row))
    #print(rdict)
    name = rdict['requester']
    timestamp = rdict['Timestamp']
    names += name
    del rdict['response status']
    ldict += [rdict]
    dfr = df_request_new[df_request_new.Timestamp == timestamp]
    
    print('Dear',name+':')
    print('\n  Here are the results from your recent CMIP6 data request(s).  The master catalog, https://cmip6.storage.googleapis.com/pangeo-cmip6.csv, will be updated with the nightly build.')
    #if len(dtrouble)>=1:
    #    print('\n '+dtrouble)
    print('\n  Please note: ')
    print('      - Data for some models (e.g., CAS/FGOALS-f3-L and NUIST/NESM3) must be obtained directly from servers which are too slow or unresponsive. ')
    print('      - We exclude data with known errors (as reported at ES-DOC) from the official listing at https://errata.es-doc.org/ .')
    print('        However, data labelled status=resolved or severity=low are included in the master catalog.')
    

    print('      - Some data we have not been able to clean up enough to get it concatenated and save to zarr. Other datasets are only available for disjointed time periods.')
    print('\n  See the sample Jupyter Notebook at https://gist.github.com/naomi-henderson/ed1801d8ee8b992dda252f8b126876a5 for a quick introduction to accessing the data.')
    print('  For general xarray help, see https://xarray-contrib.github.io/xarray-tutorial/')
    print('\nFrom the folks at:\n  The Climate Data Science Lab\n  Division of Ocean and Climate Physics\n  LDEO/Columbia University')
    print('\n--------------------------')

    print('\nrequest:')
    display(rdict)

    print('\nresponse:')
    #try:
    #    print('old stores deleted:\n',num_removed,'\n')
    #    print('new stores added:\n',len(dz),'\n')
    #except:
    #    print(f'no new data available at ESGF API search node {ESGF_site}')

    #print('\n',dfr,len(df_master_new))
    table = response(dfr,df_master_new)

    print("\navailable data:\n  this includes your new stores but does not include datasets marked 'onhold', 'wontfix' or 'new' in the ES-DOC ERRATA")
    display(table)
    print('\n\n')

In [ ]:
assert False

In [ ]:
! mv csv/request_new.csv csv/requests.csv

In [ ]:
#xr.open_zarr(fs.get_mapper(gsurl),consolidated=True)
#xr.open_zarr(zbdir,consolidated=True)